reading **Foundations for Analysis with Python**

这篇笔记主要讲csv文件的读取和写入，还有读取后对数据进行的处理。在上一篇笔记里，用两段代码作为例子简单说明了文本文件的读写，回顾请戳[chpOneDataContainer.ipynb](https://github.com/QLWeilcf/cunyu/blob/master/chpOneDataContainer.ipynb)，csv也是一种文本文件，《Python数据分析基础》中说：
> CSV（comma-separated value，逗号分隔值）文件格式是一种非常简单的数据存储与分享
方式。CSV 文件将数据表格存储为纯文本，表格（或电子表格）中的每个单元格都是一个
数值或字符串。

具体关于csv的介绍可以参考[百科](https://baike.so.com/doc/5126285-5355574.html)。

程序处理的一般为内存中的数据，这样程序运行完之后数据就消失了，因此一般对于重要的数据会将其保存为文件等物理格式。文件的读写对于数据分析是非常重要的，《Python数据分析基础》的第二章通过一些具体的代码讲了csv文件的读写，整体思路为：普通方式读写csv文件、用csv库读写、用pandas库读写、读写数据进入内存后的数据筛选和统计量计算。本篇笔记沿用这个思路。

## 目录

- 普通方式读写csv
- 用csv库读写
- pandas库读写
- 读入后的数据处理


### 普通方式读写
读写文件的整体流程是：**打开文件-->按行读入数据-->全部或部分保存到容器中-->关闭文件-->对内存中的数据进行分析**。为了表述和调用的方便，下面的演示代码多打包为函数。
本篇笔记使用的数据可以在[《Python数据分析基础》GitHub](https://github.com/cbrownley/foundations-for-analytics-with-python/blob/master/csv/supplier_data.csv)获得。主要用到的csv文件名为：supplier_data.csv

在Excel 2016中打开和Notepad++中打开效果分别为：

![supplier_data_pic](https://github.com/QLWeilcf/LcfsPythonWork/blob/master/readingForDS/dsIpynbPic/supplier_data_pic.png)

In [13]:
#普通方式读取supplier_data.csv 并写入一个叫 s_csv_data.csv的文件
def generalWayRcsv(in_fname,out_fname):
    #传入的参数分别为输入的文件路径，输出文件路径
    with open(in_fname,'r',encoding='UTF-8') as freader:  #读入
        with open(out_fname,'w') as fwriter:
            header = freader.readline() #读取首行
            header = header.strip() 
            header_list = header.split(',') 
            print(header_list) 
            fwriter.write(','.join(map(str,header_list))+'\n') 
            for row in freader: 
                row = row.strip() 
                row_list = row.split(',') 
                print(row_list) 
                fwriter.write(','.join(map(str,row_list))+'\n')
    print('end')
in_f="supplier_data.csv"
out_f="s_csv_data.csv"
generalWayRcsv(in_f,out_f) #如果报错注意路径或文件编码问题

['Supplier Name', 'Invoice Number', 'Part Number', 'Cost', 'Purchase Date']
['Supplier X', '001-1001', '2341', '$500.00', '1/20/14']
['Supplier X', '001-1001', '2341', '$500.00', '1/20/14']
['Supplier X', '001-1001', '5467', '$750.00', '1/20/14']
['Supplier X', '001-1001', '5467', '$750.00', '1/20/14']
['Supplier Y', '50-9501', '7009', '$250.00', '1/30/14']
['Supplier Y', '50-9501', '7009', '$250.00', '1/30/14']
['Supplier Y', '50-9505', '6650', '$125.00', '2/3/14']
['Supplier Y', '50-9505', '6650', '$125.00', '2/3/14']
['Supplier Z', '920-4803', '3321', '$615.00', '2/3/14']
['Supplier Z', '920-4804', '3321', '$615.00', '2/10/14']
['Supplier Z', '920-4805', '3321', '$615.00', '2/17/14']
['Supplier Z', '920-4806', '3321', '$615.00', '2/24/14']
end


从以上代码可以看出读取csv的流程为：打开文件-->按行读入数据并按逗号切分字符串为列表(-->写入数据)

### 用csv库读写
csv是一个内置的Python库，通过 import csv引用。它可以很好地处理csv文件。

In [14]:
import csv
def csvLibRWdata(in_fname,out_fname):
    with open(in_fname, 'r', newline='') as csv_in_file: 
        with open(out_fname, 'w', newline='') as csv_out_file: 
            filereader = csv.reader(csv_in_file, delimiter=',') 
            filewriter = csv.writer(csv_out_file, delimiter=',') 
            for row_list in filereader:
                print(row_list)
                filewriter.writerow(row_list)
                
in_f="supplier_data.csv"
out_f="s_csv_data.csv"
csvLibRWdata(in_f,out_f) 

['Supplier Name', 'Invoice Number', 'Part Number', 'Cost', 'Purchase Date']
['Supplier X', '001-1001', '2341', '$500.00', '1/20/14']
['Supplier X', '001-1001', '2341', '$500.00', '1/20/14']
['Supplier X', '001-1001', '5467', '$750.00', '1/20/14']
['Supplier X', '001-1001', '5467', '$750.00', '1/20/14']
['Supplier Y', '50-9501', '7009', '$250.00', '1/30/14']
['Supplier Y', '50-9501', '7009', '$250.00', '1/30/14']
['Supplier Y', '50-9505', '6650', '$125.00', '2/3/14']
['Supplier Y', '50-9505', '6650', '$125.00', '2/3/14']
['Supplier Z', '920-4803', '3321', '$615.00', '2/3/14']
['Supplier Z', '920-4804', '3321', '$615.00', '2/10/14']
['Supplier Z', '920-4805', '3321', '$615.00', '2/17/14']
['Supplier Z', '920-4806', '3321', '$615.00', '2/24/14']


从以上代码可以看出，csv库可以简化很多语句，简化整体代码，通过csv.reader进行读取，并且可以指定分隔符的样式，就算是用空格隔开的文本文件（有些csv或txt文件用这种方式保存数据）可以通过改变csv.reader的参数来读文件，方便了很多。

### pandas库读写

pandas是数据分析经常接触到的库，pandas和numpy、matplotlib是Python数据分析工具的三驾马车。功能很强大，Anaconda集成环境是包含了这三个库的，如果需要自己安装可以参考[安装并使用python pandas 百度经验](https://jingyan.baidu.com/article/c275f6ba17d179e33d756721.html) 等教程。Python3环境下有pip可以一条命令行安装需要的第三方库的，本处不展开。

In [15]:
import pandas as pd
def pandasLibRWdata(in_fname,out_fname):
    data=pd.read_csv(in_fname)
    data_column_by_name = data.loc[:, ['Invoice Number', 'Purchase Date']] 
    print(data)
    data_column_by_name.to_csv(out_fname, index=False)
    
in_f="supplier_data.csv"
out_f="s_csv_data.csv"
pandasLibRWdata(in_f,out_f)    

   Supplier Name Invoice Number  Part Number     Cost Purchase Date
0     Supplier X       001-1001         2341  $500.00       1/20/14
1     Supplier X       001-1001         2341  $500.00       1/20/14
2     Supplier X       001-1001         5467  $750.00       1/20/14
3     Supplier X       001-1001         5467  $750.00       1/20/14
4     Supplier Y        50-9501         7009  $250.00       1/30/14
5     Supplier Y        50-9501         7009  $250.00       1/30/14
6     Supplier Y        50-9505         6650  $125.00        2/3/14
7     Supplier Y        50-9505         6650  $125.00        2/3/14
8     Supplier Z       920-4803         3321  $615.00        2/3/14
9     Supplier Z       920-4804         3321  $615.00       2/10/14
10    Supplier Z       920-4805         3321  $615.00       2/17/14
11    Supplier Z       920-4806         3321  $615.00       2/24/14


从上面的代码来看，pandas作业一个数据分析库，实现csv饿读写也是很简洁的，data是一个dataframe（可以翻译作数据框）数据结构。pandas库通过read_csv(in_fname)进行csv文件读取，to_csv()可以将一个dataframe数据写入到csv，很方便。

### 读入后的数据处理
得到的数据一般保存到列表(上一篇笔记有较详细的讲解)或者dataframe里，就可以用if进行相关筛选或者进行平均数、众数、方差等的计算了，当然对于文本数据不能计算这些，需要根据csv中具体的数据类型编写合适的代码。

In [16]:
#将上面的代码扩展为读取和处理
import pandas as pd
def pandasLibPredata(in_fname):
    data=pd.read_csv(in_fname) #读入数据
    p_sum=sum(data['Part Number'])
    p_mean=data['Part Number'].mean()
    print(p_sum)
    print(p_mean)
    
in_f="supplier_data.csv"
pandasLibPredata(in_f)

56218
4684.833333333333


下一篇笔记会讲解Excel文件的读写和读入数据后的处理，仍然用到 pandas 库，并且还会用到 xlrd 和 xlwt 两个库。要熟练文件读写需要多练习，有任何建议欢迎联系本人。